## 准备数据

In [11]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
import numpy as np

def mnist_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # 归一化
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    return train_loader, test_loader

In [12]:
class MyModel(nn.Module):
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(784, 256)
        self.layer2 = nn.Linear(256, 128)
        self.layer3 = nn.Linear(128, 10)
        
    def forward(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = x.view(-1, 784)
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return self.layer3(x)

model = MyModel()

optimizer = optim.Adam(model.parameters())

## 计算 loss

In [13]:
from torch.nn import functional as F

# 定义损失计算
def compute_loss(logits, labels):
    return F.cross_entropy(logits, labels)

# 定义准确率计算
def compute_accuracy(logits, labels):
    predictions = logits.argmax(dim=1, keepdim=True).squeeze()
    correct = predictions.eq(labels.view_as(predictions)).sum().item()
    accuracy = correct / len(labels)
    return accuracy

# 定义单步训练过程
def train_one_step(model, optimizer, x, y):
    model.train()
    optimizer.zero_grad()
    logits = model(x)
    loss = compute_loss(logits, y)
    loss.backward()
    optimizer.step()
    accuracy = compute_accuracy(logits, y)
    return loss.item(), accuracy

In [14]:
# 定义训练和测试函数
def train_one_epoch(epoch, model, optimizer, data_loader):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for X, y in data_loader:
        loss, accuracy = train_one_step(model, optimizer, X, y)
        total_loss += loss * X.size(0)
        total_correct += accuracy * X.size(0)
        total_samples += X.size(0)
    print(f'Epoch: {epoch}, Training: Loss {total_loss / total_samples}, Accuracy {total_correct / total_samples}')

def test(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for X, y in data_loader:
            logits = model(X)
            test_loss += compute_loss(logits, y).item() * X.size(0)
            correct += (logits.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= len(data_loader.dataset)
    accuracy = correct / len(data_loader.dataset)
    print(f'Test Loss: {test_loss}, Test Accuracy: {accuracy}')

### 实际训练与测试

In [16]:
train_loader, test_loader = mnist_dataset()

# 每个epoch（整个测试集）训练一次
for epoch in range(0, 10):
    train_one_epoch(epoch, model, optimizer, train_loader)

test(model, test_loader)

Epoch: 0, Training: Loss 0.07856199843777964, Accuracy 0.9747666666666667
Epoch: 1, Training: Loss 0.06713325431081467, Accuracy 0.97815
Epoch: 2, Training: Loss 0.060229327476909386, Accuracy 0.9806166666666667
Epoch: 3, Training: Loss 0.056767978312191554, Accuracy 0.98175
Epoch: 4, Training: Loss 0.052404679072772464, Accuracy 0.9825666666666667
Epoch: 5, Training: Loss 0.047222310828578584, Accuracy 0.9845166666666667
Epoch: 6, Training: Loss 0.04487329669001629, Accuracy 0.9857166666666667
Epoch: 7, Training: Loss 0.03952409928478592, Accuracy 0.9869666666666667
Epoch: 8, Training: Loss 0.040703595763954215, Accuracy 0.9872666666666666
Epoch: 9, Training: Loss 0.035657177080215964, Accuracy 0.9882333333333333
Test Loss: 0.09386542378545941, Test Accuracy: 0.9777
